# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [1]:
# TODOS: 
# 1) import any other libraries you might need
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

In [2]:
# 2) instantiate a Spark session 
spark = SparkSession \
        .builder \
        .appName("5_dataframe_quiz") \
        .getOrCreate()

In [3]:
# 3) read in the data set located at the path "data/sparkify_log_small.json"
df = spark.read.json("data/sparkify_log_small.json")

In [4]:
# 4) check dataset
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Question 1

Which page did user id "" (empty string) NOT visit?

In [16]:
# TODO: write your code to answer question 1
df.select("page").where('userId == ""').dropDuplicates().sort("page").show()

+-----+
| page|
+-----+
|About|
| Help|
| Home|
|Login|
+-----+



In [64]:
# filter for users with blank user id
blank_pages = df.filter(df.userId == '') \
    .select('page') \
    .dropDuplicates()

# get a list of possible pages that could be visited
all_pages = df.select('page').dropDuplicates()

# find values in all_pages that are not in blank_pages
# these are the pages that the blank user did not go to
for row in set(all_pages.collect()) - set(blank_pages.collect()):
    print(row.page)

Downgrade
Settings
Save Settings
NextSong
Submit Downgrade
Logout
Submit Upgrade
Error
Upgrade


# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [18]:
# TODO: use this space to explore the behavior of the user with an empty string
df.select('method','page','registration','song','status','level').where('userId == ""').show()

+------+-----+------------+----+------+-----+
|method| page|registration|song|status|level|
+------+-----+------------+----+------+-----+
|   PUT|Login|        null|null|   307| free|
|   GET| Home|        null|null|   200| paid|
|   GET| Home|        null|null|   200| free|
|   GET| Home|        null|null|   200| paid|
|   PUT|Login|        null|null|   307| paid|
|   PUT|Login|        null|null|   307| free|
|   GET| Home|        null|null|   200| paid|
|   GET| Home|        null|null|   200| paid|
|   PUT|Login|        null|null|   307| paid|
|   GET| Home|        null|null|   200| paid|
|   PUT|Login|        null|null|   307| paid|
|   GET| Home|        null|null|   200| paid|
|   GET| Help|        null|null|   200| paid|
|   GET| Home|        null|null|   200| paid|
|   PUT|Login|        null|null|   307| paid|
|   GET| Home|        null|null|   200| free|
|   PUT|Login|        null|null|   307| free|
|   GET| Home|        null|null|   200| free|
|   PUT|Login|        null|null|  

# Question 3

How many female users do we have in the data set?

In [28]:
# TODO: write your code to answer question 3
df_f = df.where('gender == "F"')
df_f.select('userId').dropDuplicates().count()

462

# Question 4

How many songs were played from the most played artist?

In [88]:
# TODO: write your code to answer question 4
art = df.groupby('artist').count().sort(desc('count'))
art.show(2)

+--------+-----+
|  artist|count|
+--------+-----+
|    null| 1653|
|Coldplay|   83|
+--------+-----+
only showing top 2 rows



SyntaxError: keyword can't be an expression (<ipython-input-79-e98670907a59>, line 1)

# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [91]:
# TODO: write your code to answer question 5
from pyspark.sql.window import Window
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())

user_window = Window \
    .partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

cusum = df.filter((df.page == 'NextSong') | (df.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', Fsum('homevisit').over(user_window))

cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+



In [100]:
cumsum()

NameError: name 'cumsum' is not defined